In [1]:
import numpy as np 
import matplotlib.pyplot as plt
 
def carry(state, theta):
	'''
	carry U operatior on state
	suppose U=exp(-i*theta/2*sigmay)
	'''
	U = np.array([[np.cos(theta/2), -np.sin(theta/2)],[np.sin(theta/2), np.cos(theta/2)]])
	final_state = np.dot(U, state)

	return final_state

def blue_carry(state, theta):
	'''
	blue carry U on state
	suppose U=exp(-i*theta/2*sigmay)
	and |0>-->|1>,phonon number plus 1
	|1>-->|0>, phonon number minus 1
	'''
	# U operator on up and down level of state
	state_up = np.vstack((state[0], np.zeros(len(state[0]))))
	state_down = np.vstack((np.zeros(len(state[0])), state[1]))
	U = np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])
	carry_state_up = np.dot(U, state_up)
	carry_state_down = np.dot(U, state_down)
	# after U operation, down level minus 1
	up_state = carry_state_up[0]
	down_state_minus = carry_state_up[1]
	down_state_minus_0 = state_up[0][0]  # blue carry can not drive the phonon=0 && up level
	down_state_minus = np.delete(down_state_minus, 0)
	down_state_minus = np.insert(down_state_minus, len(down_state_minus)-1, 0)
	# after U operation, up level plus 1
	up_state_plus = carry_state_down[0]
	down_state = carry_state_down[1]
	up_state_plus = np.insert(up_state_plus, 0, down_state_minus_0)
	up_state_plus = np.delete(up_state_plus, len(up_state_plus)-1)
	# probability add up and down level
	up = up_state + up_state_plus
	down = down_state_minus + down_state
	final_state = np.vstack((up, down))

	return final_state

def red_carry(state, theta):
	'''
	red carry U on state
	suppose U=exp(-i*theta/2*sigmay)
	and |0>-->|1>,phonon number minus 1
	|1>-->|0>, phonon number plus 1
	'''
	# U operator on up and down level of state
	state_up = np.vstack((state[0], np.zeros(len(state[0]))))
	state_down = np.vstack((np.zeros(len(state[0])), state[1]))
	U = np.array([[np.cos(theta/2), -np.sin(theta/2)],[np.sin(theta/2), np.cos(theta/2)]])
	carry_state_up = np.dot(U, state_up)
	carry_state_down = np.dot(U, state_down)
	# after U operation, up level minus 1
	up_state_minus = carry_state_down[0]
	down_state = carry_state_down[1]
	up_state_minus_0 = state_down[1][0] # red carry can not drive the phonon=0 && down level
	up_state_minus = np.delete(up_state_minus,0)
	up_state_minus = np.insert(up_state_minus,len(up_state_minus)-1,0)
	# after U operation, down level plus 1
	up_state = carry_state_up[0]
	down_state_plus = carry_state_up[1]
	down_state_plus = np.insert(down_state_plus,0,up_state_minus_0)
	down_state_plus = np.delete(down_state_plus,len(down_state_plus)-1)
	# probability add up and down level
	up = up_state + up_state_minus
	down = down_state_plus + down_state
	final_state = np.vstack((up, down))

	return final_state

def Operator(up_state, down_state, func, theta):
	'''
	operator include carry, blue and red
	'''
	state = np.vstack((up_state, down_state))
	state = func(state, theta)

	return state[0], state[1]

def one_step(state, theta_1, theta_2):
	'''
	one step of complete quantum walk
	may with the help of auxiliary level
	input state of three level
	output state of three level
	T1:up-->right
	t2:down-->left 
	T2R2T1R1
	'''
	up_state = state[0]
	down_state = state[1]
	aux_state = state[2]
	up_state, down_state = Operator(up_state, down_state, carry, theta_1)
	aux_state, down_state = Operator(aux_state, down_state, carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, red_carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, theta_2)
	up_state, down_state = Operator(up_state, down_state, carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, blue_carry, np.pi)

	state = np.vstack((up_state, down_state, aux_state))
	return state

def one_step_2(state, theta_1, theta_2):
	'''
	one step of complete quantum walk
	may with the help of auxiliary level
	input state of three level
	output state of three level
	T1:up-->right
	t2:down-->left 
	T1R1T2R2
	'''
	up_state = state[0]
	down_state = state[1]
	aux_state = state[2]
	up_state, down_state = Operator(up_state, down_state, carry, theta_2)
	aux_state, down_state = Operator(aux_state, down_state, red_carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, theta_1)
	up_state, down_state = Operator(up_state, down_state, blue_carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, np.pi)

	state = np.vstack((up_state, down_state, aux_state))
	return state

def one_step_3(state, theta_1, theta_2):
	'''
	one step of complete quantum walk
	may with the help of auxiliary level
	input state of three level
	output state of three level
	T1:up-->right
	t2:down-->left 
	r1T2R2T1r1   r1=sqrt(R1)
	'''
	up_state = state[0]
	down_state = state[1]
	aux_state = state[2]
	up_state, down_state = Operator(up_state, down_state, carry, theta_1/2)
	aux_state, down_state = Operator(aux_state, down_state, carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, red_carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, theta_2)
	up_state, down_state = Operator(up_state, down_state, carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, blue_carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, carry, theta_1/2)

	state = np.vstack((up_state, down_state, aux_state))
	return state

def one_step_4(state, theta_1, theta_2):
	'''
	one step of complete quantum walk
	may with the help of auxiliary level
	input state of three level
	output state of three level
	T1:up-->right
	t2:down-->left 
	r2T1R1T2r2  r2=sqrt(R2)
	'''
	up_state = state[0]
	down_state = state[1]
	aux_state = state[2]
	up_state, down_state = Operator(up_state, down_state, carry, theta_2/2)
	aux_state, down_state = Operator(aux_state, down_state, red_carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, theta_1)
	up_state, down_state = Operator(up_state, down_state, blue_carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, carry, theta_2/2)

	state = np.vstack((up_state, down_state, aux_state))
	return state

def one_step_5(state, theta_1, theta_2):
	'''
	conventional quantum walk with boundary
	'''
	up_state = state[0]
	down_state = state[1]
	aux_state = state[2]
	up_state, down_state = Operator(up_state, down_state, carry, theta_1)
	aux_state, down_state = Operator(aux_state, down_state, red_carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, carry, np.pi)
	up_state, down_state = Operator(up_state, down_state, blue_carry, np.pi)
	aux_state, down_state = Operator(aux_state, down_state, carry, np.pi)

	state = np.vstack((up_state, down_state, aux_state))
	return state

def N_step(state, theta_1, theta_2, N):
	'''
	N step of complete quantum walk
	'''
	for i in range(N):
		state = one_step_2(state, theta_1, theta_2)

	return state



# Parameters: M(phonon number space),n(inial place)
M = 100
n = 1
N = 50
up_state = [0]*M
down_state = [0]*M
aux_state = [0]*M
up_state[0] = 1/np.sqrt(2)
down_state[1]=-1j/np.sqrt(2)
np.array(up_state)
np.array(down_state)
np.array(aux_state)
init_state = np.vstack((up_state, down_state, aux_state))

# up_state = [0]*M
# down_state = [0]*M
# aux_state = [0]*M
# up_state[0] = 0
# down_state[1]=1
# np.array(up_state)
# np.array(down_state)
# np.array(aux_state)
# init_state = np.vstack((up_state, down_state, aux_state))


# step of walk
theta_1 = np.linspace(0,4*np.pi,100)
theta_2 = np.pi/2
p=np.zeros(100)

for i,theta in enumerate(theta_1):
    state = N_step(init_state, theta, theta_2, N=50)
    P = abs(state[0])**2 + abs(state[1])**2 + abs(state[2])**2
    p[i] = P[0]
    

plt.plot(theta_1,p)
    
# state = N_step(init_state, theta_1, theta_2, N=50)
# plot result
# P = abs(state[0])**2 + abs(state[1])**2 + abs(state[2])**2
# phonon = range(len(P))
# # plt.scatter(phonon, P, s=1000*P)
# print('total pro', sum(P))
# plt.bar(range(len(P)),P)

# plt.title('$\varepsilon$',fontsize=20)

# plt.figure(figsize=(20,20))
# for i,theta in enumerate(theta_1):
#     state = N_step(init_state, theta, theta_2, N=N)
#     P = abs(state[0])**2 + abs(state[1])**2 + abs(state[2])**2
#     plt.subplot(6,4,i+1)
#     plt.xlabel('theta=%f'%theta)
#     plt.bar(range(5),P[:5])
    


# plot thermal picture
# array=np.zeros((N,M))
# for i in range(0,N):
# 	state = N_step(init_state, theta_1, theta_2, N=i)
# 	P = abs(state[0])**2 + abs(state[1])**2 + abs(state[2])**2
# 	array[i]=P
# plt.figure()
# plt.xlim(0,N)
# plt.xlabel('phonon')
# plt.ylabel('step')
# plt.imshow(array)
# plt.colorbar()



# theta=np.linspace(0*np.pi,4*np.pi,100)
# P_0=[]
# P_1=[]
# P_2=[]
# P_3=[]
# P_4=[]
# for x in theta:
#     state = N_step(init_state, x, theta_2, N=N)
#     P = abs(state[0])**2 + abs(state[1])**2 + abs(state[2])**2
#     P_0=np.append(P_0,P[0])
#     P_1=np.append(P_1,P[1])
#     P_2=np.append(P_2,P[2])
#     P_3=np.append(P_3,P[3])
#     P_4=np.append(P_4,P[4])
# plt.subplot(2,3,1)
# plt.plot(theta,P_0)
# plt.subplot(2,3,2)
# plt.plot(theta,P_1)
# plt.subplot(2,3,3)
# plt.plot(theta,P_2)
# plt.subplot(2,3,4)
# plt.plot(theta,P_3)
# plt.subplot(2,3,5)
# plt.plot(theta,P_4)

plt.show()

<Figure size 640x480 with 1 Axes>

In [2]:
# from moviepy.video.io.bindings import mplfig_to_npimage
# import moviepy.editor as mpy
# import matplotlib.pyplot as plt
# import numpy as np

# duration=10
# fig_mpl,ax = plt.subplots(1,figsize=(5,3),facecolor='white')
# phonon = np.arange(0,100)
# def transit(State):
#     P=abs(State[0])**2 + abs(State[1])**2 + abs(State[2])**2
#     return P
# P = lambda d : transit(N_step(init_state, theta_1, theta_2, N=d))
# ax.set_title("first gif")
# ax.set_xlim(-5,50)
# # print (type(phonon),type(P(0)[:50]))
# line, = ax.plot(phonon,P(0),lw=3)
 
# def make_frame_mpl(t):
#     t = int(t)
#     line.set_ydata(P(5*t))
#     return mplfig_to_npimage(fig_mpl)

# animation = mpy.VideoClip(make_frame_mpl, duration=duration)
# animation.write_gif("sinc_mpl3.gif",fps=5)

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual 
import ipywidgets as widgets 

import matplotlib.pyplot as plt 
%matplotlib inline 

import numpy as np 

def plot_func(freq): 
    x = np.linspace(0, 2*np.pi) 
    y = np.sin(x * freq) 
    plt.plot(x, y) 

interact(plot_func, freq = widgets.FloatSlider(value=5, 
               min=1, 
               max=5.0, 
               step=0.5)) 

interactive(children=(FloatSlider(value=5.0, description='freq', max=5.0, min=1.0, step=0.5), Output()), _dom_…

<function __main__.plot_func(freq)>

In [4]:
4.3/np.pi

1.3687325105902999

In [64]:
from ipywidgets import interact, interactive, fixed, interact_manual 
import ipywidgets as widgets 

import matplotlib.pyplot as plt 
%matplotlib inline 

import numpy as np 


M = 500
N = 90
up_state = [0]*M
down_state = [0]*M
aux_state = [0]*M


# np.array(up_state)
# np.array(down_state)
# np.array(aux_state)
# init_state = np.vstack((up_state, down_state, aux_state))

plt.figure()

def plot_func(n,theta_1,theta_2,theta):
#     up_state[1] = np.cos(theta)
#     down_state[0] = np.sin(theta)
    down_state[0] = 1
    np.array(up_state)
    np.array(down_state)
    np.array(aux_state)
    init_state = np.vstack((up_state, down_state, aux_state))
    n = int(n)
    state = N_step(init_state, theta_1, theta_2, N=n)
    P = abs(state[0])**2 + abs(state[1])**2 + abs(state[2])**2
    phonon = np.arange(0, len(P)) 
    plt.bar(phonon ,P) 
    plt.xlim(0,70)
    plt.ylim(0,1)

interact(plot_func, n = widgets.FloatSlider(value=40, 
               min=0, 
               max=300, 
               step=1),theta_1 = widgets.FloatSlider(value=np.pi, 
               min=-2*np.pi, 
               max=2*np.pi, 
               step=0.05*np.pi),theta_2 =widgets.FloatSlider(value=0.2, 
               min=-2*np.pi, 
               max=2*np.pi, 
               step=0.02), theta =widgets.FloatSlider(value=-np.pi/2, 
               min=-np.pi, 
               max=np.pi, 
               step=0.05)) 
plt.show()

<Figure size 432x288 with 0 Axes>

interactive(children=(FloatSlider(value=40.0, description='n', max=300.0, step=1.0), FloatSlider(value=3.14159…

![](./Phase.png)

In [23]:
state = N_step(init_state, 0, np.pi, N=3)
state[1][28]

0.57735026918962584

In [37]:
-np.pi/3

-1.0471975511965976